# CNN Model Dot Product Size Requirement Generator


Function interpret a keras model to extract each layer information such as
filter or kernel dimensions, input dimensions, output dimensions, dot product compution requirement etc and creates a csv file that can be provided to simulators like **B_ONN_SIM(Binary Optical Neural Network Simulator)**   

In [13]:
#@title
def save_models_info(model_dict):
  
  
  layers_data = [] 
  for model_name in model_dict.keys():
    
    model = model_dict[model_name]
    collect_layer_info = ['Conv2D','SeparableConv2D','Dense','DepthwiseConv2D','MaxPooling2D']
    # model.summary()
    
    for layer in model.layers:
      config = layer.get_config()
      input_info = layer.input_spec
      input_shape = layer.input_shape
      output_shape = layer.output_shape
      layer_name = layer.__class__.__name__
      if layer_name not in collect_layer_info:
          continue
      if layer_name != 'Dense':
            
            if isinstance(input_shape,list):
              input_height = input_shape[0][1]
              input_width = input_shape[0][2]
              input_depth = input_shape[0][3]
            else: 
              input_height = input_shape[1]
              input_width = input_shape[2]
              input_depth = input_shape[3]
              
            if isinstance(output_shape,list):
              output_height = output_shape[0][1]
              output_width = output_shape[0][2]
              output_depth = output_shape[0][3] 
            else: 
              output_height = output_shape[1]
              output_width = output_shape[2]
              output_depth = output_shape[3]
      else:
          output_height = output_shape[1]
          output_width = 1
          output_depth = 1

      # print(input_height)
      # print(input_shape)
      # print(output_shape)
      # print(config)
      layer_tensor_info = {}
      # print(layer.__class__.__name__)
      try:
          if config['trainable']:
            
            if layer_name in collect_layer_info:  
              # print(layer_name)
              # print(config)
              print(config)
              if layer_name != 'Dense' and layer_name != 'MaxPooling2D':
                kernel_size = config['kernel_size']
                name = layer_name
                depth = list(input_info.axes.values())[0]
                height = kernel_size[0] 
                width = kernel_size[1]
              elif layer_name == 'MaxPooling2D':
                name = layer_name
                pool_dim = config['pool_size']
                depth = 1
                width = pool_dim[0]
                height = pool_dim[1]
              else:
                name = layer_name
                depth = 1
                width = 1
                height = input_shape[1]
              if layer_name == 'DepthwiseConv2D':
                  layer_tensor_info = {}
                  # print("DepthWise Convolution")
                  layer_tensor_info['model_name'] = model_name
                  layer_tensor_info['layer_name'] = "DepthWiseConv"
                  layer_tensor_info['kernel_height'] = height
                  layer_tensor_info['kernel_width'] = width
                  layer_tensor_info['kernel_depth'] = 1
                  layer_tensor_info['tensor_count'] = depth
                  layer_tensor_info['input_shape'] = input_shape
                  layer_tensor_info['output_shape'] = output_shape
                  layer_tensor_info['tensor_shape'] = (layer_tensor_info['kernel_height'],layer_tensor_info['kernel_width'],layer_tensor_info['kernel_depth'])
                  layer_tensor_info['input_height'] = input_height
                  layer_tensor_info['input_width'] = input_width
                  layer_tensor_info['input_depth'] = input_depth 
                  layer_tensor_info['output_height'] = output_height
                  layer_tensor_info['output_width'] = output_width
                  layer_tensor_info['output_depth'] = output_depth 
                  
                  layers_data.append(layer_tensor_info)
                  # print(layer_tensor_info)
              if layer_name == 'SeparableConv2D':
                layer_tensor_info = {}
                filters = config['filters']
                # seperating into depthwise and pointwise convolutions
                layer_tensor_info['model_name'] = model_name
                layer_tensor_info['layer_name'] = "DepthWiseConv"
                layer_tensor_info['kernel_height'] = height
                layer_tensor_info['kernel_width'] = width
                layer_tensor_info['kernel_depth'] = 1
                layer_tensor_info['tensor_count'] = depth
                layer_tensor_info['input_shape'] = input_shape
                layer_tensor_info['output_shape'] = "TBF"
                layer_tensor_info['tensor_shape'] = (layer_tensor_info['kernel_height'],layer_tensor_info['kernel_width'],layer_tensor_info['kernel_depth'])
                layer_tensor_info['input_height'] = input_height
                layer_tensor_info['input_width'] = input_width
                layer_tensor_info['input_depth'] = input_depth 
                layer_tensor_info['output_height'] = output_height
                layer_tensor_info['output_width'] = output_width
                layer_tensor_info['output_depth'] = output_depth 
                layers_data.append(layer_tensor_info)
                layer_tensor_info = {}
                layer_tensor_info['model_name'] = model_name
                layer_tensor_info['layer_name'] = "PointWiseConv"
                layer_tensor_info['kernel_height'] = 1
                layer_tensor_info['kernel_width'] = 1
                layer_tensor_info['kernel_depth'] = depth
                layer_tensor_info['tensor_count'] = filters
                layer_tensor_info['input_shape'] = "TBF"
                layer_tensor_info['output_shape'] = output_shape 
                layer_tensor_info['tensor_shape'] = (layer_tensor_info['kernel_height'],layer_tensor_info['kernel_width'],layer_tensor_info['kernel_depth']) 
                layer_tensor_info['input_height'] = input_height
                layer_tensor_info['input_width'] = input_width
                layer_tensor_info['input_depth'] = input_depth 
                layer_tensor_info['output_height'] = output_height
                layer_tensor_info['output_width'] = output_width
                layer_tensor_info['output_depth'] = output_depth 
                layers_data.append(layer_tensor_info)
              if layer_name == 'Conv2D':
                layer_tensor_info = {}
                # print("Layer", layer_name)
                filters = config['filters']
                layer_tensor_info['model_name'] = model_name
                layer_tensor_info['layer_name'] = layer_name
                layer_tensor_info['kernel_depth'] = depth
                layer_tensor_info['kernel_height'] = height
                if height == 1 and width == 1:
                  layer_tensor_info['name'] = 'PointWiseConv'
                layer_tensor_info['kernel_width'] = width
                layer_tensor_info['tensor_count'] = filters
                layer_tensor_info['input_shape'] = input_shape
                layer_tensor_info['output_shape'] = output_shape
                layer_tensor_info['tensor_shape'] = (layer_tensor_info['kernel_height'],layer_tensor_info['kernel_width'],layer_tensor_info['kernel_depth'])
                layer_tensor_info['input_height'] = input_height
                layer_tensor_info['input_width'] = input_width
                layer_tensor_info['input_depth'] = input_depth 
                layer_tensor_info['output_height'] = output_height
                layer_tensor_info['output_width'] = output_width
                layer_tensor_info['output_depth'] = output_depth 
                layers_data.append(layer_tensor_info)
              if layer_name == 'Dense':
                layer_tensor_info = {}
                # print("Layer", layer_name)
                layer_tensor_info['model_name'] = model_name
                layer_tensor_info['layer_name'] = layer_name
                layer_tensor_info['kernel_depth'] = depth
                layer_tensor_info['kernel_height'] = height
                layer_tensor_info['kernel_width'] = width
                layer_tensor_info['tensor_count'] = 1
                layer_tensor_info['input_shape'] = input_shape
                layer_tensor_info['output_shape'] = output_shape
                layer_tensor_info['tensor_shape'] = (layer_tensor_info['kernel_height'],layer_tensor_info['kernel_width'],layer_tensor_info['kernel_depth'])
                layer_tensor_info['input_height'] = input_height
                layer_tensor_info['input_width'] = input_width
                layer_tensor_info['input_depth'] = input_depth 
                layer_tensor_info['output_height'] = output_height
                layer_tensor_info['output_width'] = output_width
                layer_tensor_info['output_depth'] = output_depth 
                # print("Added Dense Layer")
                layers_data.append(layer_tensor_info)
              if layer_name == 'MaxPooling2D':
                # print("Pooling Layer")
                layer_tensor_info['model_name'] = model_name
                layer_tensor_info['layer_name'] = layer_name
                layer_tensor_info['kernel_depth'] = depth
                layer_tensor_info['kernel_height'] = height
                layer_tensor_info['kernel_width'] = width
                layer_tensor_info['tensor_count'] = 1
                layer_tensor_info['input_shape'] = input_shape
                layer_tensor_info['output_shape'] = output_shape
                layer_tensor_info['tensor_shape'] = (layer_tensor_info['kernel_height'],layer_tensor_info['kernel_width'],layer_tensor_info['kernel_depth'])
                layer_tensor_info['input_height'] = input_height
                layer_tensor_info['input_width'] = input_width
                layer_tensor_info['input_depth'] = input_depth 
                layer_tensor_info['output_height'] = output_height
                layer_tensor_info['output_width'] = output_width
                layer_tensor_info['output_depth'] = output_depth 
                layers_data.append(layer_tensor_info)
                # print(layer_tensor_info)
      except Exception as e:
              print("Not Trainable Layer",e)
      # print(layer_tensor_info)

  layers_df = pd.DataFrame(layers_data)
  layers_df.to_csv(model_name+'.csv',index=False)
 

### Importing models available in keras appilcations to extract information. 
The method can also extract any model defined with keras

In [14]:
# from keras.applications import mobilenet_v2
# from keras.applications.mobilenet import MobileNet 
# from keras.applications.xception import Xception 
# from keras.applications.mobilenet_v2 import MobileNetV2
# from keras.applications.efficientnet import EfficientNetB7
# from keras.applications.nasnet import NASNetMobile
from keras.applications.resnet import ResNet50
from keras.applications.vgg16 import VGG16
# from keras.applications.inception_v3 import InceptionV3
# from keras.applications.densenet import DenseNet121
from keras.applications.vgg19 import VGG19


import pandas as pd

# xception_model = Xception()
# mobilenet_model = MobileNet()
# mobilenet_v2_model = MobileNetV2()
# efficientnet_bo_model = EfficientNetB7()
# nasnetmobile_model = NASNetMobile()
# ShuffleNetV2_model = ShuffleNetV2(include_top=True, input_shape=(32, 32, 3),load_model=None, classes=131)
# Resnet50_model = ResNet50()
VGG16_model = VGG16()
# Inceptionv3_model = InceptionV3() 
# Densnet121_model = DenseNet121()
# VGG19_Model = VGG19()
# GoogleNet_Model = GoogLeNet()
# Alexnet_model = alexnet_model()
# Resnet_Model = ResNet18()


model_dict = {
          # "Xception": xception_model,
          # "MobileNet_V2": mobilenet_v2_model,
          # "EfficientNet_B7": efficientnet_bo_model,
          # "NASNetMobile": nasnetmobile_model,
          # "ShuffleNet_V2": ShuffleNetV2_model,
          # "ResNet50": Resnet50_model,
          "VGG16": VGG16_model,
          # "Inception_V3" : InceptionV3()
          # "DenseNet121": Densnet121_model,
          # "GoogLeNet":GoogleNet_Model,
          # "Alexnet":Alexnet_model,
    

}
save_models_info(model_dict)

{'name': 'block1_conv1', 'trainable': True, 'dtype': 'float32', 'filters': 64, 'kernel_size': (3, 3), 'strides': (1, 1), 'padding': 'same', 'data_format': 'channels_last', 'dilation_rate': (1, 1), 'groups': 1, 'activation': 'relu', 'use_bias': True, 'kernel_initializer': {'class_name': 'GlorotUniform', 'config': {'seed': None}}, 'bias_initializer': {'class_name': 'Zeros', 'config': {}}, 'kernel_regularizer': None, 'bias_regularizer': None, 'activity_regularizer': None, 'kernel_constraint': None, 'bias_constraint': None}
{'name': 'block1_conv2', 'trainable': True, 'dtype': 'float32', 'filters': 64, 'kernel_size': (3, 3), 'strides': (1, 1), 'padding': 'same', 'data_format': 'channels_last', 'dilation_rate': (1, 1), 'groups': 1, 'activation': 'relu', 'use_bias': True, 'kernel_initializer': {'class_name': 'GlorotUniform', 'config': {'seed': None}}, 'bias_initializer': {'class_name': 'Zeros', 'config': {}}, 'kernel_regularizer': None, 'bias_regularizer': None, 'activity_regularizer': None, 

In [15]:
from tabulate import tabulate

model_df = pd.read_csv('VGG16.csv')
print(tabulate(model_df, headers='keys', tablefmt='psql'))

+----+--------------+--------------+----------------+-----------------+----------------+----------------+-----------------------+-----------------------+----------------+----------------+---------------+---------------+-----------------+----------------+----------------+
|    | model_name   | layer_name   |   kernel_depth |   kernel_height |   kernel_width |   tensor_count | input_shape           | output_shape          | tensor_shape   |   input_height |   input_width |   input_depth |   output_height |   output_width |   output_depth |
|----+--------------+--------------+----------------+-----------------+----------------+----------------+-----------------------+-----------------------+----------------+----------------+---------------+---------------+-----------------+----------------+----------------|
|  0 | VGG16        | Conv2D       |              3 |               3 |              3 |             64 | (None, 224, 224, 3)   | (None, 224, 224, 64)  | (3, 3, 3)      |            22